## Hubbarnet use

In [ ]:
import torch

In [ ]:
def generate_fock_states(L, N):
    """
    Generates all Fock´s states for N particles and L sites.
    """
    def gen(current, left, sites_left):
        if sites_left == 1:
            yield current + [left]
        else:
            for i in range(left + 1):
                yield from gen(current + [i], left - i, sites_left - 1)
    
    return list(gen([], N, L))

In [ ]:
L = 6 # sitios
N = 5  # partículas

fock_basis = generate_fock_states(L, N)

focks_states = []

for state in fock_basis:
    focks_states.append(state)

torch_states = torch.tensor(focks_states,dtype=torch.float32)

In [ ]:
from quspin.operators import hamiltonian
from quspin.basis import boson_basis_1d

def exact(U):
    # Parámetros del modelo
    L = 6       # número de sitios
    Nbosons = 5 # número total de bosones
    t = 1.0     # hopping
            # interacción en el sitio

    # Base de bosones
    basis = boson_basis_1d(L, Nb=Nbosons)

    # Término de hopping (b†_i b_j + h.c.)
    hop_list = [[-t, i, (i+1)%L] for i in range(L)]
    hop_ham = [["+-", hop_list], ["-+", hop_list]]

    n_list = [[-0.5*U, i] for i in range(L)]        # n_i
    nn_list = [[0.5*U, i, i] for i in range(L)]   # n_i^2

    # Término de interacción en el sitio: U n_i(n_i - 1)
    int_ham = [["n", n_list], ["nn", nn_list]]

    # Construir el Hamiltoniano total
    static = hop_ham + int_ham
    H = hamiltonian(static, [], basis=basis, dtype=np.float64)

    # Diagonalización
    E, V = H.eigh()
    return min(E)

In [ ]:
from hubbarnet import BoseHubbardNeuralNetwork
from hubbarnet import BoseHubbardModel

network = []
exact_vals = []

for i in [2,4,6,8]:
    nn = BoseHubbardNeuralNetwork(num_sites=6,num_hidden=20)
    model = BoseHubbardModel(J=1,U=i,V=0,num_sites=6,num_particles=5,fock_states=torch_states,nn=nn)
    model.fit(100)
    
    network.append(model.groud_state_energy())
    exact_vals.append(exact(i))

In [ ]:
from matplotlib import pyplot as plt

plt.scatter([2,4,6,8],exact_vals,label="Exact diagonalization")
plt.scatter([2,4,6,8],torch.stack(network).detach().numpy(),label="Network",marker="s",facecolors='none',edgecolors='red')

plt.legend()
plt.title("Diagonalizacion exacta vs Red Neuronal")
plt.xlabel("U")
plt.ylabel("Energia base")
plt.grid(True)
plt.show()